In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn import metrics
import statsmodels.api as sm

In [2]:
df=pd.read_csv('diamonds.csv')

# Make test default model

In [3]:
#create log columns
def create_log_columns(df,param):
    cat_price=['I1', 'SI2', 'SI1', 'VS2', 'VS1', 'VVS2', 'VVS1', 'IF']
    df['clarity_log'] = df['clarity'].apply(lambda x: 1+ (cat_price.index(x)))
    color_list = sorted(list(df.color.unique()),reverse=True)
    df['color_log'] = df['color'].apply(lambda x: 1+ (color_list.index(x)))
    cut_price = ['Fair', 'Good', 'Very Good', 'Premium', 'Ideal']
    df['cut_log'] = df['cut'].apply(lambda x: 1+ (cut_price.index(x)))
    if param == 1:
        df['price_log'] = np.log(df['price'])
    df['carat_log']=np.log(df['carat'])
    df['cut_log']=np.log(df['cut_log'])
    df['color_log']=np.log(df['color_log'])
    df['clarity_log']=np.log(df['clarity_log'])
    df['depth_log']=np.log(df['depth'])
    df['table_log']=np.log(df['table'])
    df['x_log']=np.log(df['x']+1)
    df['y_log']=np.log(df['y']+1)
    df['z_log']=np.log(df['z']+1)
    return df

# Make model for categories combination

In [4]:
df = create_log_columns(df,1)
clar_lst = df.clarity.unique()
dfs = [df.query(f'(clarity =="{x}")') for x in clar_lst]

In [8]:
def ger_model(df):
    y= df['price_log']
    X= df[['carat_log','cut_log','color_log','depth_log','table_log','x_log','y_log','z_log']]
    modelo = LinearRegression()
    modelo.fit(X, y)
    return modelo

In [9]:
modelos = [ger_model(df) for df in dfs]

In [10]:
def do_predict(row):
    clar = row['clarity']
    X= [[row['carat_log'],row['cut_log'],row['color_log'],row['depth_log'],row['table_log'],row['x_log'],row['y_log'],row['z_log']]]
    index = list(clar_lst).index(clar)
    model = modelos[index] 
    y_predict = model.predict(X)
    y_predict=np.exp(y_predict)
    return y_predict
 

In [11]:
y_predict = df.apply(lambda row : do_predict(row), axis=1)
y_real = df['price']
metrics.r2_score(y_real, y_predict).round(3)

0.966

In [12]:
np.sqrt(metrics.mean_squared_error(y_real,y_predict))

740.9133966289618

In [17]:
rick_diamonds = pd.read_csv('rick_diamonds.csv')
rick_copy = rick_diamonds.copy()
rick_copy = create_log_columns(rick_copy,0)
y_predict = rick_copy.apply(lambda row : do_predict(row)[0], axis=1)
rick_diamonds['price_predicted'] = y_predict
rick_diamonds.to_csv('./Deliverables/rick_diamonds_new.csv')

In [19]:
y_predict

0        3018.650992
1         850.358886
2        1463.264087
3        7044.695870
4       18024.236560
            ...     
4995      450.635509
4996     1884.278552
4997     4671.502204
4998      843.357430
4999      755.856967
Length: 5000, dtype: float64

In [18]:
rick_diamonds

,carat,cut,color,clarity,depth,table,x,y,z,price_predicted
0,0.91,Very Good,H,SI2,63.4,57.0,6.10,6.18,3.89,3018.650992
1,0.38,Good,F,VS2,60.3,61.0,4.68,4.71,2.83,850.358886
2,0.52,Ideal,H,VS2,61.8,56.0,5.19,5.16,3.20,1463.264087
3,1.10,Ideal,G,VS1,61.3,56.0,6.65,6.69,4.09,7044.695870
4,2.28,Ideal,G,SI2,61.6,57.0,8.44,8.37,5.18,18024.236560
...,...,...,...,...,...,...,...,...,...,...
4995,0.30,Very Good,D,SI2,62.2,59.0,4.25,4.30,2.66,450.635509
4996,0.51,Ideal,F,VVS2,61.9,56.0,5.09,5.14,3.16,1884.278552
4997,1.01,Very Good,F,SI1,60.6,60.0,6.37,6.43,3.88,4671.502204
4998,0.31,Ideal,F,VVS1,62.1,56.0,4.35,4.38,2.71,843.357430
